# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846617405818                   -0.70    4.0
  2   -7.852296780074       -2.25       -1.53    1.0
  3   -7.852646093408       -3.46       -2.52    3.2
  4   -7.852646677445       -6.23       -3.36    2.0
  5   -7.852646685941       -8.07       -4.76    1.5
  6   -7.852646686725       -9.11       -5.25    3.2
  7   -7.852646686730      -11.30       -6.20    1.2
  8   -7.852646686730      -12.64       -7.60    2.8
  9   -7.852646686730      -14.45       -7.56    2.2
 10   -7.852646686730   +  -14.45       -8.41    1.8
 11   -7.852646686730   +  -15.05       -9.06    2.2
 12   -7.852646686730      -14.75       -9.33    1.0
 13   -7.852646686730      -15.05      -11.14    2.0
 14   -7.852646686730   +    -Inf      -11.01    2.2
 15   -7.852646686730   +  -15.05      -11.63    2.0
 16   -7.852646686730      -15.05      -12.15    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846858562971                   -0.70           4.2
  2   -7.852525720594       -2.25       -1.62   0.80    5.0
  3   -7.852608795800       -4.08       -2.79   0.80    5.0
  4   -7.852646567345       -4.42       -3.55   0.80    7.2
  5   -7.852646683453       -6.94       -4.55   0.80    8.8
  6   -7.852646686681       -8.49       -5.10   0.80   10.8
  7   -7.852646686725      -10.36       -6.13   0.80   11.5
  8   -7.852646686730      -11.30       -7.05   0.80   13.0
  9   -7.852646686730      -13.30       -7.76   0.80   15.0
 10   -7.852646686730   +    -Inf       -8.70   0.80   17.2
 11   -7.852646686730   +    -Inf       -9.19   0.80   18.0
 12   -7.852646686730   +  -14.57      -10.61   0.80   19.0
 13   -7.852646686730   +  -14.75      -11.23   0.80   21.5
 14   -7.852646686730      -14.75      -12.69   0.80   22.8


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.432367e+01     3.522415e+00
 * time: 0.5089669227600098
     1     1.567406e+00     1.750575e+00
 * time: 0.7573089599609375
     2    -1.824972e+00     2.210469e+00
 * time: 0.7851910591125488
     3    -3.784081e+00     1.910111e+00
 * time: 0.8239850997924805
     4    -5.481884e+00     1.756440e+00
 * time: 0.8636269569396973
     5    -7.035517e+00     9.044292e-01
 * time: 0.9019079208374023
     6    -7.381703e+00     9.582907e-01
 * time: 0.9302890300750732
     7    -7.716077e+00     5.166007e-01
 * time: 0.9575440883636475
     8    -7.798943e+00     1.855733e-01
 * time: 0.9846320152282715
     9    -7.829111e+00     1.077944e-01
 * time: 1.0111260414123535
    10    -7.843134e+00     8.259688e-02
 * time: 1.0399680137634277
    11    -7.849469e+00     3.060944e-02
 * time: 1.068356990814209
    12    -7.851617e+00     1.776809e-02
 * time: 1.0973551273345947
    13    -7.852206e+00     1.448504e-02
 * time: 1.12501311302

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846866721601                   -0.70    4.2
  2   -7.852327873613       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686705                   -2.54
  2   -7.852646686730      -10.61       -5.91
  3   -7.852646686730      -15.05      -12.61
|ρ_newton - ρ_scf|  = 4.26349504147748e-13
|ρ_newton - ρ_scfv| = 4.655949169465855e-13
|ρ_newton - ρ_dm|   = 5.774318394212327e-10
